# Project 2: Topic Classification

In this project, you'll work with text data from newsgroup postings on a variety of topics. You'll train classifiers to distinguish between the topics based on the text of the posts. Whereas with digit classification, the input is relatively dense: a 28x28 matrix of pixels, many of which are non-zero, here we'll represent each document with a "bag-of-words" model. As you'll see, this makes the feature representation quite sparse -- only a few words of the total vocabulary are active in any given document. The bag-of-words assumption here is that the label depends only on the words; their order is not important.

The SK-learn documentation on feature extraction will prove useful:
http://scikit-learn.org/stable/modules/feature_extraction.html

Each problem can be addressed succinctly with the included packages -- please don't add any more. Grading will be based on writing clean, commented code, along with a few short answers.

As always, you're welcome to work on the project in groups and discuss ideas on the course wall, but please prepare your own write-up and write your own code.

In [1]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn library for importing the newsgroup data.
from sklearn.datasets import fetch_20newsgroups

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Load the data, stripping out metadata so that we learn classifiers that only use textual features. By default, newsgroups data is split into train and test sets. We further split the test so we have a dev set. Note that we specify 4 categories to use for this project. If you remove the categories argument from the fetch function, you'll get all 20 categories.

In [2]:
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test',
                                     remove=('headers', 'footers', 'quotes'),
                                     categories=categories)

num_test = len(newsgroups_test.target)
test_data, test_labels = newsgroups_test.data[num_test/2:], newsgroups_test.target[num_test/2:]
dev_data, dev_labels = newsgroups_test.data[:num_test/2], newsgroups_test.target[:num_test/2]
train_data, train_labels = newsgroups_train.data, newsgroups_train.target

print 'training label shape:', train_labels.shape
print 'test label shape:', test_labels.shape
print 'dev label shape:', dev_labels.shape
print 'labels names:', newsgroups_train.target_names

training label shape: (2034,)
test label shape: (677,)
dev label shape: (676,)
labels names: ['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']


(1) For each of the first 5 training examples, print the text of the message along with the label.

In [3]:
def P1(num_examples=5):
### STUDENT START ###
    for i in range(num_examples):
        print "\nEaxmple %i,  Label: %s" % (i+1, train_labels[i])
        print "Text: \n%s" % train_data[i]

### STUDENT END ###
P1()


Eaxmple 1,  Label: 1
Text: 
Hi,

I've noticed that if you only save a model (with all your mapping planes
positioned carefully) to a .3DS file that when you reload it after restarting
3DS, they are given a default position and orientation.  But if you save
to a .PRJ file their positions/orientation are preserved.  Does anyone
know why this information is not stored in the .3DS file?  Nothing is
explicitly said in the manual about saving texture rules in the .PRJ file. 
I'd like to be able to read the texture rule information, does anyone have 
the format for the .PRJ file?

Is the .CEL file format available from somewhere?

Rych

Eaxmple 2,  Label: 3
Text: 


Seems to be, barring evidence to the contrary, that Koresh was simply
another deranged fanatic who thought it neccessary to take a whole bunch of
folks with him, children and all, to satisfy his delusional mania. Jim
Jones, circa 1993.


Nope - fruitcakes like Koresh have been demonstrating such evil corruption
for centuries.

Ea

(2) Use CountVectorizer to turn the raw training text into feature vectors. You should use the fit_transform function, which makes 2 passes through the data: first it computes the vocabulary ("fit"), second it converts the raw text into feature vectors using the vocabulary ("transform").

The vectorizer has a lot of options. To get familiar with some of them, write code to answer these questions:

a. The output of the transform (also of fit_transform) is a sparse matrix: http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.sparse.csr_matrix.html. What is the size of the vocabulary? What is the average number of non-zero features per example? What fraction of the entries in the matrix are non-zero? Hint: use "nnz" and "shape" attributes.

b. What are the 0th and last feature strings (in alphabetical order)? Hint: use the vectorizer's get_feature_names function.

c. Specify your own vocabulary with 4 words: ["atheism", "graphics", "space", "religion"]. Confirm the training vectors are appropriately shaped. Now what's the average number of non-zero features per example?

d. Instead of extracting unigram word features, use "analyzer" and "ngram_range" to extract bigram and trigram character features. What size vocabulary does this yield?

e. Use the "min_df" argument to prune words that appear in fewer than 10 documents. What size vocabulary does this yield?

f. Using the standard CountVectorizer, what fraction of the words in the dev data are missing from the vocabulary? Hint: build a vocabulary for both train and dev and look at the size of the difference.

In [11]:
def P2():
### STUDENT START ###

    # (a.) Turn the raw training text into feature vectors
    count_vect = CountVectorizer()
    X_train = count_vect.fit_transform(train_data)
    # Size of the vocabulary
    [num_examples, num_features] = X_train.shape
    print "Size of Vocabulary: %i" % num_features
    # The average number of non-zero features per example
    print "Average number of non-zero features per example %2.1f" % (X_train[0].nnz / float(num_examples))
    # The fraction of the entries in the matrix are non-zero
# TODO: how is this different? 
    print "The fraction of non-zero entries in the matrix: %2.1f" % (X_train[1].nnz / float(num_features))
    
    # (b.) Print the 0th and last feature strings (in alphabetical order). 
    print "\nFeature Strings: "
    print "\tFirst: %s" % count_vect.get_feature_names()[0]
    print "\tLast:  %s" % count_vect.get_feature_names()[-1]
    feature_names = count_vect.get_feature_names()
    
    # (c.) Specify your own vocabulary with 4 words: ["atheism", "graphics", "space", "religion"]. 
    my_vocab = ['atheism', 'graphics', 'space', 'religion']
    cv_vocab = CountVectorizer(vocabulary=my_vocab)
    X_train_vocab = cv_vocab.fit_transform(train_data)
    [l, w] = X_train_vocab.shape
    print "\nSpecifing Vocab List:"
    print my_vocab
    print "Size of Vocabulary with specified vocab list: %i " % w
    print "Average number of non-zero features per example %2.1f" % (X_train_vocab.nnz / float(l))


    # (d.) Extract Bigram and Trigram Character features 
# TODO: does this mean charaters or words
# TODO: Are we to include single feautres too
    bigram_vectorizer = CountVectorizer(ngram_range=(2, 3))
    analyze = bigram_vectorizer.build_analyzer()
    X_train_bigram = bigram_vectorizer.fit_transform(train_data)
    [l, w] = X_train_bigram.shape
    print "\nSize of Bigram and trigram Vocabulary: %i" % w
    feat_names_bigram = bigram_vectorizer.get_feature_names()
    

    # (e.) Prune for words that appear in fewer than 10 documents
# TODO: Are we Which spefic arguments are we to use for this answer 
    bigram_vectorizer = CountVectorizer(ngram_range=(2, 3), min_df = 10)
    analyze = bigram_vectorizer.build_analyzer()
    X_train_bigram_pruned = bigram_vectorizer.fit_transform(train_data)
    [l, w] = X_train_bigram_pruned.shape
    print "\nSize of Bigram and trigram Vocabulary After Pruning: %i" % w
    

    # (f.) Determine Words from Dev not in Train 
    train_vocab = count_vect.vocabulary_.keys()
    cv_dev = CountVectorizer()
    X_dev = cv_dev.fit_transform(dev_data)
    dev_vocab = cv_dev.vocabulary_.keys()
    yes, no = 0, 0
    for word in dev_vocab:
        if word not in train_vocab:
            no += 1
        else: 
            yes += 1
    print "\n%1.2f Words in Dev Vocab are not in Train Vocab" % (no / float(len(dev_vocab)))

    return X_train, feature_names, X_train_bigram, feat_names_bigram
### STUDENT END ###
[X_train, feature_names, X_train_bigram, feat_names_bigram] = P2()

Size of Vocabulary: 26879
Average number of non-zero features per example 0.0
The fraction of non-zero entries in the matrix: 0.0

Feature Strings: 
	First: 00
	Last:  zyxel

Specifing Vocab List:
['atheism', 'graphics', 'space', 'religion']
Size of Vocabulary with specified vocab list: 4 
Average number of non-zero features per example 0.3

Size of Bigram and trigram Vocabulary: 510583

Size of Bigram and trigram Vocabulary After Pruning: 3381

0.25 Words in Dev Vocab are not in Train Vocab


(3) Use the default CountVectorizer options and report the f1 score (use metrics.f1_score) for a k nearest neighbors classifier; find the optimal value for k. Also fit a Multinomial Naive Bayes model and find the optimal value for alpha. Finally, fit a logistic regression model and find the optimal value for the regularization strength C using l2 regularization. A few questions:

a. Why doesn't nearest neighbors work well for this problem?

b. Any ideas why logistic regression doesn't work as well as Naive Bayes?

c. Logistic regression estimates a weight vector for each class, which you can access with the coef\_ attribute. Output the sum of the squared weight values for each class for each setting of the C parameter. Briefly explain the relationship between the sum and the value of C.

In [6]:
# Defining Function to fit models with specific parameters to avoid duplicating code
# Used in P3
def fit_model(model_name, search_grid, score):
    model = model_name()
    param_searcher = GridSearchCV(model, search_grid, cv=10, scoring=score)
    param_searcher.fit(X_train, train_labels)
    return param_searcher

In [32]:
def P3(score, neighbors, alphas, regularization_cs):
### STUDENT START ###
# TODO: Add plots for grid searces of params 
    # Fit Models and Calculate scores

    # k-NN
    knn = fit_model(KNeighborsClassifier, neighbors, score)
   
    # Multinomial Naive Bayes
    mnb = fit_model(MultinomialNB, alphas, score)
    
    # Logistic Regression 
    lr = fit_model(LogisticRegression, regularization_Cs, score)
    
    
    # Report Results
    print "K-Nearest Neighbors"
    print "\tOptimale k = %i" % knn.best_params_.get('n_neighbors')
    print "\tBest %s Score: %0.3f" % (score, knn.best_score_)
    
    print "\nMultinomial Naive Bayes"
    print "\tOptimale alpha = %1.4f" % mnb.best_params_.get('alpha')
    print "\tBest %s Score: %0.3f" % (score, mnb.best_score_)
    
    print "\nLogistic Regression"
    print "\tOptimale alpha = %i" % lr.best_params_.get('C')
    print "\tBest %s Score: %0.3f" % (score, lr.best_score_)
    
    return lr.best_params_ 

# Set model initilaization parameters
# TODO: create better search parameters
score = 'f1_weighted'
neighbors = {'n_neighbors': np.linspace(1, 150, 150, endpoint=True)}
alphas = {'alpha': [0.0, 0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0, 50.0, 100.0, 500.0, 1000.0]}
# alphas = {'alpha': np.linspace(0, 100, 1000, endpoint=True)}
# regularization_Cs = {'C': [1, 10, 100, 1000], 'penalty': ['l2']}
regularization_Cs = {'C': np.linspace(1, 150, 150, endpoint=True), 'penalty': ['l2']}

### STUDENT END ###
lr_best_params = P3(score, neighbors, alphas, regularization_Cs)

KeyboardInterrupt: 

ANSWER:

a. Why doesn't nearest neighbors work well for this problem? 

b. Any ideas why logistic regression doesn't work as well as Naive Bayes?

c. Logistic regression estimates a weight vector for each class, which you can access with the coef_ attribute. Output the sum of the squared weight values for each class for each setting of the C parameter. Briefly explain the relationship between the sum and the value of C. 

In [ ]:
lr.coef_

(4) Train a logistic regression model. Find the 5 features with the largest weights for each label -- 20 features in total. Create a table with 20 rows and 4 columns that shows the weight for each of these features for each of the labels. Create the table again with bigram features. Any surprising features in this table?

In [29]:
def P4(feature_names, lr_best_params):
### STUDENT START ###
    
    # Train Model For Original Model
    lr = LogisticRegression(C=lr_best_params.get('C'), penalty=lr_best_params.get('penalty'))
    lr.fit(X_train, train_labels)
    coeffs = lr.coef_
    
    # Print Weights and Features for each Category Label
    print "Original Features with highest weights for each label"
    print "Index  |  Feature Name        | %15s | %20s | %15s | %15s |" % (categories[0], categories[1], categories[2], categories[3])
    for i in range(coeffs.shape[0]):
        idx = np.argsort(np.abs(coeffs[i]))
        for j in idx[0:5]:
            print "%6i | %20s | %15s | %20s | %15s | %15s |" % (j, feature_names[j], coeffs[0,j], coeffs[1,j], coeffs[2,j], coeffs[3,j])

            
    # Train Model For Bigram Features Model
    # Search for best regularization C parameter 
    regularization_Cs = {'C': [1, 10, 100, 1000], 'penalty': ['l2']} 
    lr = LogisticRegression()
    param_searcher = GridSearchCV(lr, regularization_Cs, cv=10, scoring=score)
    param_searcher.fit(X_train_bigram, train_labels)
    lrbigram_best_params = param_searcher.best_params_
    
    lr_bigram = LogisticRegression(C=lrbigram_best_params.get('C'), penalty=lrbigram_best_params.get('penalty'))
    lr_bigram.fit(X_train_bigram, train_labels)
    coeffs_bigram = lr_bigram.coef_
    
    # Print Weights and Features for each Category Label
    print "\n\nBigram Features with highest weights for each label"
    print "Index  |  Feature Name        | %15s | %20s | %15s | %15s |" % (categories[0], categories[1], categories[2], categories[3])
    for i in range(coeffs_bigram.shape[0]):
        idx = np.argsort(np.abs(coeffs_bigram[i]))
        for j in idx[0:5]:
             print "%6i | %20s | %15s | %20s | %15s | %15s |" % (j, feat_names_bigram[j], coeffs_bigram[0,j], coeffs_bigram[1,j], coeffs_bigram[2,j], coeffs_bigram[3,j])
   
    
    
### STUDENT END ###
P4(feature_names, lr_best_params)

Original Features with highest weights for each label
Index  |  Feature Name        |  alt.atheism |   talk.religion.misc | comp.graphics|    sci.space |
 17376 |                  oed | -5.80296171209e-09 |    2.66618691247e-05 | 1.8145407176e-05 | -2.70139471871e-05 |
  4085 |                  axe | -5.80296171209e-09 |    2.66618691247e-05 | 1.8145407176e-05 | -2.70139471871e-05 |
 18299 |             perjurer | -5.80296171209e-09 |    2.66618691247e-05 | 1.8145407176e-05 | -2.70139471871e-05 |
 20543 |            rephrased | -5.80296171209e-09 |    2.66618691247e-05 | 1.8145407176e-05 | -2.70139471871e-05 |
 22660 |           specifying | -5.80296171209e-09 |    2.66618691247e-05 | 1.8145407176e-05 | -2.70139471871e-05 |
 10462 |               fitted | -0.00765021711194 |    1.10159458579e-09 | 1.41210381115e-05 | 7.16274903227e-06 |
 17995 |            parallels | -0.00260470666119 |    -1.1577801793e-08 | -1.04796380186e-05 | 0.00213403225992 |
 24498 |              touring | 5.63

ANSWER:

(5) Try to improve the logistic regression classifier by passing a custom preprocessor to CountVectorizer. The preprocessing function runs on the raw text, before it is split into words by the tokenizer. Your preprocessor should try to normalize the input in various ways to improve generalization. For example, try lowercasing everything, replacing sequences of numbers with a single token, removing various other non-letter characters, and shortening long words. If you're not already familiar with regular expressions for manipulating strings, see https://docs.python.org/2/library/re.html, and re.sub() in particular. With your new preprocessor, how much did you reduce the size of the dictionary?

For reference, I was able to improve dev F1 by 2 points.

In [ ]:
# porter steming to adjust word choice
# trying to make features and adjust words to match more likely 
# take out puntuation and contractions

In [ ]:
#def empty_preprocessor(s):
#    return s

#def better_preprocessor(s):
### STUDENT START ###

### STUDENT END ###

#def P5():
### STUDENT START ###

### STUDENT END ###
#P5()

(6) The idea of regularization is to avoid learning very large weights (which are likely to fit the training data, but not generalize well) by adding a penalty to the total size of the learned weights. That is, logistic regression seeks the set of weights that minimizes errors in the training data AND has a small size. The default regularization, L2, computes this size as the sum of the squared weights (see P3, above). L1 regularization computes this size as the sum of the absolute values of the weights. The result is that whereas L2 regularization makes all the weights relatively small, L1 regularization drives lots of the weights to 0, effectively removing unimportant features.

Train a logistic regression model using a "l1" penalty. Output the number of learned weights that are not equal to zero. How does this compare to the number of non-zero weights you get with "l2"? Now, reduce the size of the vocabulary by keeping only those features that have at least one non-zero weight and retrain a model using "l2".

Make a plot showing accuracy of the re-trained model vs. the vocabulary size you get when pruning unused features by adjusting the C parameter.

Note: The gradient descent code that trains the logistic regression model sometimes has trouble converging with extreme settings of the C parameter. Relax the convergence criteria by setting tol=.01 (the default is .0001).

In [ ]:
def P6():
    # Keep this random seed here to make comparison easier.
    np.random.seed(0)

    ### STUDENT START ###
    # fit with all features
    lr_l1 = LogisticRegression(C=100, penalty="l1")
    lr_l1.fit(X_train, train_labels)
    # coefs = lr_l1.coef_
    # Output the number of learned weights that are not equal to zero
    # How does this compare to the number of non-zero weights you get with "l2"? 

    # reduce the size of the vocabulary by keeping only those features that have at 
    # least one non-zero weight and retrain a model using "l2"
    # X_train_reduce = 
    
    # Retrain with penalty = "l2"
    lr_l1 = LogisticRegression(C=100, penalty="l2")
    lr_l1.fit(X_train_reduce, train_labels)
    
    
    # Make a plot showing accuracy of the re-trained model vs. the vocabulary size you get when pruning unused 
    # features by adjusting the C parameter
    
    ### STUDENT END ###
P6()

(7) Use the TfidfVectorizer -- how is this different from the CountVectorizer? Train a logistic regression model with C=100.

Make predictions on the dev data and show the top 3 documents where the ratio R is largest, where R is:

maximum predicted probability / predicted probability of the correct label

What kinds of mistakes is the model making? Suggest a way to address one particular issue that you see.

In [ ]:
#def P7():
### STUDENT START ###

## STUDENT END ###
#P7()

ANSWER:

(8) EXTRA CREDIT

Try implementing one of your ideas based on your error analysis. Use logistic regression as your underlying model.